In [37]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

af_dir = '../../all_faces_bucket/'
disk_data_dir = '../../all_faces_disk/home/jupyter/forensics_split/'
!pip install git+https://github.com/qubvel/efficientnet
!pip install --user deepstack==0.0.9

import efficientnet.tfkeras
from deepstack.base import KerasMember
from deepstack.ensemble import DirichletEnsemble
from deepstack.ensemble import StackEnsemble
import sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import StackingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
# !sudo kill -9 PID # clear GPU memory where 9 is PID number
# !watch -n0.1 nvidia-smi

  Cloning https://github.com/qubvel/efficientnet to /tmp/pip-req-build-n8cbxo8o
  Running command git clone -q https://github.com/qubvel/efficientnet /tmp/pip-req-build-n8cbxo8o
  Created wheel for efficientnet: filename=efficientnet-1.1.0-py3-none-any.whl size=18397 sha256=6cc01d39f1da0656f0d26d7d8fd43806208977efbb482186b8b89e7a8c087f7f
  Stored in directory: /tmp/pip-ephem-wheel-cache-wv6e1_59/wheels/11/69/85/814d64d694c96db0eef17b718042d644a1e54f113920481920
Successfully built efficientnet
  Using cached scipy-1.4.1-cp37-cp37m-manylinux1_x86_64.whl (26.1 MB)
ERROR: tensorflow-serving-api 2.1.0 has requirement tensorflow~=2.1.0, but you'll have tensorflow 2.2.0 which is incompatible.
ERROR: tensorflow-model-analysis 0.22.2 has requirement pyarrow<0.17,>=0.16, but you'll have pyarrow 0.17.1 which is incompatible.
ERROR: tensorflow-data-validation 0.22.2 has requirement joblib<0.15,>=0.12, but you'll have joblib 0.15.1 which is incompatible.
ERROR: tensorflow-data-validation 0.22.2 has

# Load models

In [11]:
def get_model(config):
    '''Loads one of the saved models based on specified config'''
    
    return load_model(af_dir + 'trained_models/saved_models/config' + str(models_to_load[0]) + '.h5')

model1 = get_model(0)
model2 = get_model(1)
# model3 = get_model(2)
# model4 = get_model(3)
# model5 = get_model(4)
# model6 = get_model(5)
# model7 = get_model(6)
# model8 = get_model(7)
# model9 = get_model(8)
# model10 = get_model(9)
# model11 = get_model(10)
# model11 = get_model(11)

# Create image generators

In [17]:
# Create data generators
def augment_data(directory, batch):
    '''Prepares train-time augmentation using given training and validations data)
    
    Returns train_data, val_data'''

    datagen_train = ImageDataGenerator(
            featurewise_center=False,  # set input mean to 0 over the dataset
            samplewise_center=True,  # set each sample mean to 0
            featurewise_std_normalization=False,  # divide inputs by std of the dataset
            samplewise_std_normalization=True,  # divide each input by its std
            zca_whitening=False,  # apply ZCA whitening
            zca_epsilon=1e-06,  # epsilon for ZCA whitening
            rotation_range=20,  # randomly rotate images in the range (degrees, 0 to 180)
            shear_range=0.,  # set range for random shear
            zoom_range=0.,  # set range for random zoom
            channel_shift_range=0.,  # set range for random channel shifts
            # set mode for filling points outside the input boundaries
            fill_mode='nearest',
            cval=0.,  # value used for fill_mode = "constant"
            brightness_range=None,
            horizontal_flip=True,  # randomly flip images
            vertical_flip=False,  # randomly flip images
            # set rescaling factor (applied before any other transformation)
            rescale=None,
            # set function that will be applied on each input
#             preprocessing_function=deepaug.joint_function,
            # image data format, either "channels_first" or "channels_last"
            data_format=None,
            # fraction of images reserved for validation (strictly between 0 and 1)
            )
    
    datagen_test = ImageDataGenerator(samplewise_center=True, samplewise_std_normalization=True)

    # Classes give the folders storing the two different categories
    train_data = datagen_train.flow_from_directory(directory + '/train',
                                             target_size=(224,224), batch_size = batch)
    
    val_data = datagen_test.flow_from_directory(directory + '/validation',
                                             target_size=(224,224), batch_size = batch)
    
    return train_data, val_data

train_data, val_data = augment_data('../../forensics_split', batch=32)

Found 37124 images belonging to 2 classes.
Found 37201 images belonging to 2 classes.


# Instantiate KerasMembers
This will find all training and validation predictions. A fit method can then be applied to obtain to the members to create the ensemble.

In [19]:
member1 = KerasMember(name="model1", keras_model=model1, train_batches=train_data, val_batches=val_data)
member2 = KerasMember(name="model2", keras_model=model2, train_batches=train_data, val_batches=val_data)
# member3 = KerasMember(name="model3", keras_model=model3, train_batches=train_data, val_batches=val_data)
# member4 = KerasMember(name="model4", keras_model=model4, train_batches=train_data, val_batches=val_data)

Instructions for updating:
Please use Model.predict, which supports generators.
1163/1163 [==============================] - 163s 140ms/step


# Fit weighted average ensemble

In [23]:
wAvgEnsemble = DirichletEnsemble()
wAvgEnsemble.add_members([member1, member2])
wAvgEnsemble.fit()
wAvgEnsemble.describe()

model1 - Weight: 0.3436 - roc_auc_score: 0.5000
model2 - Weight: 0.6564 - roc_auc_score: 0.5000
DirichletEnsemble roc_auc_score: 0.5000


# Fit stacking ensemble

In [36]:
stack = StackEnsemble()
stack.model = RandomForestRegressor(verbose=0, n_estimators=200, 
                                  max_depth=15, n_jobs=20, min_samples_split=20)
stack.add_members([member1, member2])
stack.fit()
stack.describe(metric = accuracy_score)

Calling predict
model1 - accuracy_score: 0.5000
model2 - accuracy_score: 0.5000


ValueError: Classification metrics can't handle a mix of binary and continuous targets

In [39]:
#Ensure you have the scikit-learn version >= 0.22 installed
print("sklearn version must be >= 0.22. You have:", sklearn.__version__)

stack = StackEnsemble()

# 2nd Level Meta-Learner
estimators = [
    ('rf', RandomForestClassifier(verbose=0, n_estimators=200, max_depth=15, n_jobs=20, min_samples_split=30)),
    ('etr', ExtraTreesClassifier(verbose=0, n_estimators=200, max_depth=10, n_jobs=20, min_samples_split=20))
]
# 3rd Level Meta-Learner
clf = StackingClassifier(
    estimators=estimators, final_estimator=LogisticRegression()
)

stack.model = clf
stack.add_members([member1, member2])
stack.fit()
stack.describe(metric=sklearn.metrics.accuracy_score)

sklearn version must be >= 0.22. You have: 0.23.1
Calling predict
model1 - accuracy_score: 0.5000
model2 - accuracy_score: 0.5000
StackEnsemble accuracy_score: 0.6664


0.6663530550254025

In [ ]:
member1.train_probs